In [7]:
# install necessary libraries
%pip --quiet install bs4 langchainhub langchain_chroma
%pip --quiet install pypdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
# Connect to openAI and get llm
import os

os.environ["OPENAI_API_KEY"] = ''

from langchain_openai import ChatOpenAI # type: ignore

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [9]:
# import lib
from langchain import hub # type: ignore
from langchain.chains import create_retrieval_chain # type: ignore
from langchain.chains.combine_documents import create_stuff_documents_chain # type: ignore
from langchain_chroma import Chroma # type: ignore
from langchain_community.document_loaders import PyPDFLoader # type: ignore
from langchain_core.output_parsers import StrOutputParser # type: ignore
from langchain_core.prompts import ChatPromptTemplate # type: ignore
from langchain_core.runnables import RunnablePassthrough # type: ignore
from langchain_openai import OpenAIEmbeddings # type: ignore
from langchain_text_splitters import RecursiveCharacterTextSplitter # type: ignore

In [4]:
# Load the PDF file of "The Jaunt" by Stephen King
loader = PyPDFLoader("The_Jaunt-Stephen_King.pdf")
docs = loader.load()

# Split the loaded document into chunks of 1000 characters with an overlap of 200 characters
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create a vector store from the split documents using OpenAI embeddings
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Set up a retriever to find and use the most relevant snippets from the document
retriever = vectorstore.as_retriever()

In [5]:
# Compile the system prompt for the assistant to use when answering questions
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

# Create a chat prompt template that includes the system prompt and the user's input
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Create a chain that feeds the prompt to the large language model (LLM) to generate responses
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Create a retrieval chain to fetch tokenized documents and integrate it with the question-answer chain
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Invoke the retrieval and question-answer process with the input "who is patty?"
response = rag_chain.invoke({"input": "who is patty?"})

# Print the assistant's answer
print(response["answer"])

Patty is a character mentioned in the retrieved context, who is Mark's daughter. She is described as being nervous and scared about the Jaunt, an instantaneous teleportation process. Patty's curiosity about the Jaunt and its effects on consciousness is highlighted in the text.


In [6]:
# Invoke the retrieval and question-answer process with the input "What is the story about?"
response = rag_chain.invoke({"input": "What is the story about?"})

# Print the assistant's answer
print(response["answer"])

The story is about Victor Carune, who conducted an experiment involving mice to discover a problem. Carune got excited about his findings and rushed back to his laboratory, almost wrecking his truck on the way. He realized that the mice showed him there was a problem, leading to his breakthrough in the research.
